# Vizsga osszefoglalo


A gyakorlati vizsgara a kovetkezo dolgokat kell tudni:
- Tarolt eljarasok (Stored Procedures): 
    - letrehozas
    - meghivasa
    - parameterek
    - eredmenyek 
- Tranzakciok
    - letrehozas (begin transaction)
    - commit 
    - rollback
- Hibakezeles
    - try-catch
    - throw : 
    - raiserror
    - @@ERROR - valtozo
    - hibak lekerdezese es megjelenitese (ERROR_MESSAGE(), ERROR_NUMBER(), ERROR_SEVERITY(), ERROR_STATE(), ERROR_LINE(), ERROR_PROCEDURE())
- Kurzorok
    - deklaralasa 
    - nyitasa
    - lekerdezese (bejarasa) 
    - zarasa


## Feladat

Toltsuk le a [Mozi scriptet](./Szeminarium_12/Mozi.sql) a Szeminarium_12 konyvtarbol.



Irjunk tarolt eljarast amely vetiteseket vezet be a Vetites tablaba. A tarolt eljaras parameterei a kovetkezok legyenek:   
 - Idopont: DATETIME tipusu, a vetites idopontja
 - VetitesekSzama: INT tipusu kimeneti parameter, a felvezetett vetitesek szama

A tarolt eljaras a kovetkezo feladatokat vegzi el:
- Ellenorzi, hogy a VetitesNapja parameterben megadott idopont a jelenlegi idopont utan van-e. Ha nem, akkor a VetitesekSzama parameterben adjon vissza 0-t es -1-et teritsen vissza a tarolt eljaras.
- Veszi a Filmek tablabol azokat a filmeket, amelyek Aktivak es ellenorzi, hogy van-e mar vetitesuk a Vetites tablaban a VetitesNapja parameterben megadott idoponton. Ha igen akkor nem kell felvezetni a filmet, ha nem akkor fel kell vezetni a filmet a Vetites tablaba.  
- A vetites terme: a film hossza alapjan a kovetkezo szabaly alapjan kerul kiszamitasra, viszont csak akkor ha a teremben meg nincs masik vetites abban az idopontban, parameterben megadott idoponton. Ha van akkor a kovetkezo terembe kerul a vetites:
    - 60 perc alatt 1-es terem
    - 60-120 perc kozott 2-es terem
    - 120-180 perc kozott 3-as terem
    - 180-240 perc kozott 4-es terem
    - 240 perc felett 5-os terem
- A vetites ara, a film legutobbi vetitesenek az ara ha mar volt vetitve kulonben a film csillagai alapjan a kovetkezo szabaly alapjan kerul kiszamitasra:
    - 1 csillag: 10 lej
    - 2 csillag: 15 lej
    - 3 csillag: 20 lej
    - 4 csillag: 25 lej
    - 5 csillag: 50 lej

- A VetitesekSzama parameterben adja vissza, hogy hany vetitest vezetett be a tarolt eljaras, sikeretelen futas eseten -1-et adjon vissza

A tarolt eljaras neve: dbo.VetitesFelvezetese

A teljes pontszam erdekeben hasznaljunk kurzorokat.


## A feladat megoldas lepesei
- [ ]  A tarolt eljaras letrehozasa megfelelo parameterekkel
- [ ]  Deklaraljuk esetleges parameterek ertekeit valamint tranzakciot inditunk
- [ ]  A tarolt eljaras ellenorzi, hogy a VetitesNapja parametert
- [ ]  A tarolt eljaras veszi a filmeket, amelyek Aktivak es bejarja oket kurzorral
- [ ]  A tarolt eljaras ellenorzi, hogy van-e mar vetitesuk a Vetites tablaban a VetitesNapja parameterben megadott idoponton
- [ ]  A tarolt eljaras kiszamitja a vetites termer es arat
- [ ]  A tarolt eljaras felvezeti a vetitest a Vetites tablaba 
- [ ]  Ellenorzi a beszuras sikeressseget az @@ERROR valtozo ertekevel - hiba eseten visszavonja a tranzakciot (Hasznalhatjuk a RAISERROR vagy TRY/CATCH-THROW parancsot is ) siker eseten noveli a VetitesekSzama parametert
- [ ] Sikeres futas eseten COMMIT parancsot hasznalunk 








In [24]:
CREATE OR ALTER PROCEDURE spVETITESFELVEZETESE
@DATUM DATETIME,
@VETITESEKSZAMA INT OUTPUT

AS
BEGIN
    SET NOCOUNT ON;
    SET TRANSACTION ISOLATION LEVEL SERIALIZABLE
    BEGIN TRAN
        BEGIN TRY
            SELECT @VETITESEKSZAMA = 0
            IF @DATUM < GETDATE()
                BEGIN
                    RAISERROR('A vetites nem lehet korabbi, mint a mai nap!', 16, 1)
                    RETURN -1
                END
            
                DECLARE @fIlmID INT,@AR INT,@TEREM INT
                                            
 
                SELECT @AR = 1, @TEREM = @VETITESEKSZAMA + 1 -- TODO: TEREM ID 
                DECLARE fIlmIDCursor CURSOR FOR
                SELECT FilmID FROM Filmek WHERE AKTIV = 1
                OPEN fIlmIDCursor
                FETCH NEXT FROM fIlmIDCursor INTO @fIlmID
                WHILE @@FETCH_STATUS = 0
                    BEGIN
                        -- SELECT FILMCIM FROM FILMEK WHERE FilmID = @fIlmID
                        IF NOT EXISTS (SELECT * FROM Vetites WHERE FilmID = @fIlmID AND VetitesIdopontja = @DATUM)
                            BEGIN  
                                SELECT @AR = CASE
                                                    WHEN Csillag = 5 THEN 50
                                                    WHEN Csillag = 4 THEN 40
                                                    WHEN Csillag = 3 THEN 30
                                                    WHEN Csillag = 2 THEN 20
                                                    ELSE 10
                                                    END,
                                                
                                        @TEREM = CASE 
                                        WHEN hossz < 60 THEN 1
                                        WHEN hossz < 90 THEN 2
                                        WHEN hossz < 120 THEN 3
                                        WHEN hossz < 150 THEN 4
                                        WHEN hossz < 180 THEN 5
                                        ELSE 6
                                        END
                                    FROM Filmek WHERE FilmID = @fIlmID
                                    INSERT INTO Vetites(FilmID, 
                                VetitesIdopontja, 
                                Terem, 
                                Ar) 
                                VALUES (@fIlmID,
                                 @DATUM,
                                @TEREM,
                                @AR)
                                IF @@ERROR <> 0
                                    BEGIN
                                        RAISERROR('Hiba tortent a vetites felvetele soran!', 16, 1)
                                        RETURN -1
                                    END
                                SET @VETITESEKSZAMA = @VETITESEKSZAMA + 1

                                
                            END
                        FETCH NEXT FROM fIlmIDCursor INTO @fIlmID
                    END
                CLOSE fIlmIDCursor
                DEALLOCATE fIlmIDCursor

           
                COMMIT 
        END TRY
        BEGIN CATCH
        SELECT ERROR_MESSAGE() AS ErrorMessage,
         ERROR_NUMBER() AS ErrorNumber,
         ERROR_SEVERITY() AS ErrorSeverity,
         ERROR_STATE() AS ErrorState

            ROLLBACK
        END CATCH
END

Commands completed successfully.

Total execution time: 00:00:00.003

In [19]:
DECLARE @AR int = 0,@TEREM INT = 0

SELECT @AR = CASE
                                                    WHEN Csillag = 5 THEN 50
                                                    WHEN Csillag = 4 THEN 40
                                                    WHEN Csillag = 3 THEN 30
                                                    WHEN Csillag = 2 THEN 20
                                                    ELSE 10
                                                    END,
                                                    @TEREM = CASE
                                                    WHEN Hossz > 60 THEN 1
                                                    ELSE 2
                                                    END
                                    FROM Filmek WHERE FilmID = 1
SELECT @AR,@TEREM

(1 row affected)

Total execution time: 00:00:00.008

(No column name),(No column name)
50,1


In [11]:
SELECT * FROM Filmek WHERE Aktiv = 1

(3 rows affected)

Total execution time: 00:00:00.011

FilmID,FilmCim,StudioID,MufajID,Csillag,MegjEv,Hossz,Aktiv
1,Les Miserables,1,2,5,2012,158,1
6,Ticket to paradise,5,4,3,2022,104,1
7,No time to die,5,3,5,2021,163,1


In [26]:
DECLARE @OUT INT,@RETURN INT 
EXEC @RETURN = spVETITESFELVEZETESE '2024-12-13 18:00:00.000', @out OUTPUT
SELECT @RETURN AS ReturnValue, @OUT AS 'Out'
select * from Vetites

(1 row affected)

(16 rows affected)

Total execution time: 00:00:00.013

ReturnValue,Out
0,3


VetitesID,FilmID,VetitesIdopontja,Terem,Ar
1,1,2012-12-12 18:00:00.000,3,10.00
2,1,2018-12-12 18:00:00.000,6,15.00
3,1,2019-12-12 18:00:00.000,7,18.00
4,1,2019-11-12 16:30:00.000,3,14.00
5,2,2019-11-12 18:00:00.000,1,15.00
6,4,2019-11-12 22:15:00.000,4,16.00
7,6,2022-09-16 18:00:00.000,2,21.00
8,6,2022-09-16 22:00:00.000,2,24.00
9,6,2022-09-22 20:00:00.000,5,25.00
10,6,2022-09-22 22:00:00.000,2,25.00


In [2]:
SELECT * FROM Vetites

(10 rows affected)

Total execution time: 00:00:00.024

VetitesID,FilmID,VetitesIdopontja,Terem,Ar
1,1,2012-12-12 18:00:00.000,3,10.00
2,1,2018-12-12 18:00:00.000,6,15.00
3,1,2019-12-12 18:00:00.000,7,18.00
4,1,2019-11-12 16:30:00.000,3,14.00
5,2,2019-11-12 18:00:00.000,1,15.00
6,4,2019-11-12 22:15:00.000,4,16.00
7,6,2022-09-16 18:00:00.000,2,21.00
8,6,2022-09-16 22:00:00.000,2,24.00
9,6,2022-09-22 20:00:00.000,5,25.00
10,6,2022-09-22 22:00:00.000,2,25.00
